In [2]:
import ee
import geemap

# ee.Authenticate()
ee.Initialize()

In [26]:
extent_path = fr"D:\Users\FRAN\Documents\IPA\GUI_project\bago_bb.geojson"

AOI = geemap.geojson_to_ee(extent_path)
START_DATE = '2025-11-06'
END_DATE = '2025-11-30'
CLOUD_FILTER = 60
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST_km = 1
BUFFER_m = 50
output_name = 'bago_s1_20251116' 

In [27]:
def mask_edge(image):
  edge = image.lt(-30.0)
  masked_image = image.mask().And(edge.Not())
  return image.updateMask(masked_image)

collection = (
    ee.ImageCollection("COPERNICUS/S1_GRD")
      .filterBounds(AOI)   # Lisbon example
      .filterDate(START_DATE, END_DATE)
      .filter(ee.Filter.eq("instrumentMode", "IW"))
      .filter(ee.Filter.eq("orbitProperties_pass", "ASCENDING"))
      .select("VV")
)

# Extract the list of dates
dates = collection.aggregate_array("system:time_start").map(
    lambda t: ee.Date(t).format("YYYY-MM-dd")
)

# Get the result into Python
date_list = dates.getInfo()
print(date_list)

vis_params_vv = {
    'bands': ['VV'],
    'min': -25,
    'max': 5
}

m = geemap.Map()
m.centerObject(AOI, 10)
m.addLayer(collection.mean(), vis_params_vv, 'S1 VV Mean')
m.addLayer(AOI, {}, 'AOI')
m

# spring = ee.Filter.date('2015-03-01', '2015-04-20')
# late_spring = ee.Filter.date('2015-04-21', '2015-06-10')
# summer = ee.Filter.date('2015-06-11', '2015-08-31')

# desc_change = ee.Image.cat(
#     desc.filter(spring).mean(),
#     desc.filter(late_spring).mean(),
#     desc.filter(summer).mean(),
# )

# asc_change = ee.Image.cat(
#     asc.filter(spring).mean(),
#     asc.filter(late_spring).mean(),
#     asc.filter(summer).mean(),
# )

# m = geemap.Map()
# m.set_center(5.2013, 47.3277, 12)
# m.add_layer(asc_change, {'min': -25, 'max': 5}, 'Multi-T Mean ASC', True)
# m.add_layer(desc_change, {'min': -25, 'max': 5}, 'Multi-T Mean DESC', True)
# m

['2025-11-29', '2025-11-29', '2025-11-16', '2025-11-23', '2025-11-28']


Map(center=[10.50257851109158, 122.99037724315335], controls=(WidgetControl(options=['position', 'transparent_…

In [15]:
dataset = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate(START_DATE, END_DATE)
)

visualization = {
    'min': 0.0,
    'max': 10000,
    'bands': ['B4', 'B3', 'B2'],
    'gamma': 1.4,
}

m = geemap.Map()
m.centerObject(AOI, 10)
m.addLayer(dataset.median(), visualization, 'RGB')
m.addLayer(AOI, {}, 'AOI')
m

Map(center=[10.50257851109158, 122.99037724315335], controls=(WidgetControl(options=['position', 'transparent_…

In [42]:
# Function to extract dates from ImageCollection
def get_image_dates(image_collection):
    # Get the list of images as a FeatureCollection
    images = image_collection.toList(image_collection.size())
    dates = [
        ee.Image(images.get(i)).date().format("YYYY-MM-dd").getInfo()
        for i in range(images.size().getInfo())
    ]
    return dates

def get_s2_sr_cld_col(aoi, start_date, end_date, img_collection):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection(img_collection)
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))
    
def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST_km input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST_km*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER_m*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)

def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

s2_visualization = {
    'min': 0,
    'max': 10000,
    'bands': ['B4', 'B3', 'B2'],
    'gamma': 1.2
}

def compute_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)

map1 = geemap.Map(height="900px")
map1.centerObject(AOI, 10)
map1

Map(center=[10.50257851109158, 122.99037724315335], controls=(WidgetControl(options=['position', 'transparent_…

In [43]:
# APPLYING S2 CLOUD MASK
collection_name = 'COPERNICUS/S2_SR_HARMONIZED'
s2_sr_cld_col = get_s2_sr_cld_col(AOI, START_DATE, END_DATE, collection_name)
s2_sr_median = (s2_sr_cld_col.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask)
                             .median())

map1.addLayer(s2_sr_median, s2_visualization, name='s2 median cloudless')
map1.addLayer(AOI, {}, 'AOI')
# nir band
# map1.addLayer(s2_sr_median.select(['B8']), { 'min': 0, 'max': 10000, 'gamma': 1.}, name='s2 band 8/nir')

In [24]:
print('All band names:', s2_sr_median.bandNames().getInfo())

All band names: ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B11', 'B12']


ADD NDVI FOR VIZ

In [ ]:
ndvi = s2_sr_median.normalizedDifference(['B8', 'B4']).rename('NDVI')
image_wndvi = s2_sr_median.select(['B2', 'B3', 'B4']).addBands(ndvi).toFloat()

FOR L1C: SIAC atmospheric correction, cloudmasking

In [25]:
siac = ee.require('users/marcyinfeng/utils:SIAC')

AttributeError: module 'ee' has no attribute 'require'

In [26]:

# APPLYING S2 CLOUD MASK
s2_sr_cld_col = get_s2_sr_cld_col(AOI, START_DATE, END_DATE, 'COPERNICUS/S2_HARMONIZED')
s2_sr_median = (s2_sr_cld_col.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask)
                             .median())

map1.addLayer(s2_sr_median, s2_visualization, name='s2 median cloudless')

EEException: reduce.median: Error in map(ID=20200104T020439_20200104T020440_T51NZJ):
Image.select: Band pattern 'SCL' did not match any bands. Available bands: [B1, B2, B3, B4, B5, B6, B7, B8, B8A, B9, B10, B11, B12, QA10, QA20, QA60, MSK_CLASSI_OPAQUE, MSK_CLASSI_CIRRUS, MSK_CLASSI_SNOW_ICE, probability, clouds]

From vector training data,
-convert to raster
-get stratified random samples for training and validation
-use RF as classifier

Downloading S2 AOI

In [44]:
task = ee.batch.Export.image.toDrive(**{
    'image': s2_sr_median.clip(AOI).select(['B2', 'B3', 'B4', 'B8']),
    'description': 'bago_s2_202511_comp' ,
    'scale': 10,
    'region': AOI.geometry(),
    'crs': 'EPSG:4326'
    
})

task.start()
print(f"Running task {task.status()['task_type']} of {task.status()['description']}")

# while(task.status()['state'] != 'COMPLETED'):
#     continue



Running task EXPORT_IMAGE of bago_s2_202511_comp


In [46]:
task.status()


{'state': 'COMPLETED',
 'description': 'bago_s2_202511_comp',
 'priority': 100,
 'creation_timestamp_ms': 1765673421863,
 'update_timestamp_ms': 1765673707889,
 'start_timestamp_ms': 1765673431018,
 'task_type': 'EXPORT_IMAGE',
 'destination_uris': ['https://drive.google.com/'],
 'attempt': 1,
 'batch_eecu_usage_seconds': 160.400634765625,
 'id': '2CPCDYBVHUJG7QMZK7BXVOHX',
 'name': 'projects/earthengine-legacy/operations/2CPCDYBVHUJG7QMZK7BXVOHX'}

In [ ]:
task1024 = ee.batch.Export.image.toDrive(**{
    'image': collection.mean().clip(AOI),
    'description': output_name,
    'scale': 10,
    'region': AOI.geometry(),
    'crs': 'EPSG:4326'
    
})

task1024.start()
# print(f"Running task {task1024.status()['task_type']} of {task.status()['description']}")

# while(task.status()['state'] != 'COMPLETED'):
#     continue



NameError: name 'task' is not defined

In [29]:
task1024.status()

{'state': 'RUNNING',
 'description': 'bago_s1_20251116',
 'priority': 100,
 'creation_timestamp_ms': 1765721233506,
 'update_timestamp_ms': 1765721246640,
 'start_timestamp_ms': 1765721246561,
 'task_type': 'EXPORT_IMAGE',
 'attempt': 1,
 'id': 'YUWQQVEGE2FML4VYKQXKQ66Z',
 'name': 'projects/earthengine-legacy/operations/YUWQQVEGE2FML4VYKQXKQ66Z'}

Downloading from GDrive

In [24]:
# import the required libraries 
from __future__ import print_function 
import pickle 
import os.path 
import io 
import shutil 
import requests 
from mimetypes import MimeTypes 
from googleapiclient.discovery import build 
from google_auth_oauthlib.flow import InstalledAppFlow 
from google.auth.transport.requests import Request 
from googleapiclient.http import MediaIoBaseDownload, MediaFileUpload 

class DriveAPI: 
	global SCOPES 
	
	# Define the scopes 
	SCOPES = ['https://www.googleapis.com/auth/drive'] 

	def __init__(self): 
		
		# Variable self.creds will 
		# store the user access token. 
		# If no valid token found 
		# we will create one. 
		self.creds = None

		# The file token.pickle stores the 
		# user's access and refresh tokens. It is 
		# created automatically when the authorization 
		# flow completes for the first time. 

		# Check if file token.pickle exists 
		if os.path.exists('token.pickle'): 

			# Read the token from the file and 
			# store it in the variable self.creds 
			with open('token.pickle', 'rb') as token: 
				self.creds = pickle.load(token) 

		# If no valid credentials are available, 
		# request the user to log in. 
		if not self.creds or not self.creds.valid: 

			# If token is expired, it will be refreshed, 
			# else, we will request a new one. 
			if self.creds and self.creds.expired and self.creds.refresh_token: 
				self.creds.refresh(Request()) 
			else: 
				flow = InstalledAppFlow.from_client_secrets_file( 
					'gee-dl_credentials.json', SCOPES) 
				self.creds = flow.run_local_server(port=0) 

			# Save the access token in token.pickle 
			# file for future usage 
			with open('token.pickle', 'wb') as token: 
				pickle.dump(self.creds, token) 

		# Connect to the API service 
		self.service = build('drive', 'v3', credentials=self.creds) 

		# request a list of first N files or 
		# folders with name and id from the API. 
		results = self.service.files().list( 
			pageSize=100, fields="files(id, name)").execute() 
		items = results.get('files', []) 

		# print a list of files 

		print("Here's a list of files: \n") 
		print(*items, sep="\n", end="\n\n") 

	def FileDownload(self, file_id, file_name): 
		request = self.service.files().get_media(fileId=file_id) 
		fh = io.BytesIO() 
		
		# Initialise a downloader object to download the file 
		downloader = MediaIoBaseDownload(fh, request, chunksize=204800) 
		done = False

		try: 
			# Download the data in chunks 
			while not done: 
				status, done = downloader.next_chunk() 

			fh.seek(0) 
			
			# Write the received data to the file 
			with open(file_name, 'wb') as f: 
				shutil.copyfileobj(fh, f) 

			print("File Downloaded") 
			# Return True if file Downloaded successfully 
			return True
		except: 
			
			# Return False if something went wrong 
			print("Something went wrong.") 
			return False

	def FileUpload(self, filepath): 
		
		# Extract the file name out of the file path 
		name = filepath.split('/')[-1] 
		
		# Find the MimeType of the file 
		mimetype = MimeTypes().guess_type(name)[0] 
		
		# create file metadata 
		file_metadata = {'name': name} 

		try: 
			media = MediaFileUpload(filepath, mimetype=mimetype) 
			
			# Create a new file in the Drive storage 
			file = self.service.files().create( 
				body=file_metadata, media_body=media, fields='id').execute() 
			
			print("File Uploaded.") 
		
		except: 
			
			# Raise UploadError if file is not uploaded. 
			raise UploadError("Can't Upload File.") 

if __name__ == "__main__": 
	obj = DriveAPI() 
	# i = int(input("Enter your choice: 1 - Download file, 2- Upload File, 3- Exit.\n")) 
	i = 1 
	
	if i == 1: 
		f_id = input("Enter file id: ") 
		f_name = fr"{output_name}.tif" 
		obj.FileDownload(f_id, f_name) 
		
	elif i == 2: 
		f_path = input("Enter full file path: ") 
		obj.FileUpload(f_path) 
	
	else: 
		exit() 


Here's a list of files: 

{'id': '1eT4DJt-QsDQ_NyQQS0gaAphAF55qL9Ad', 'name': 'tagum_q12023_s2.tif'}
{'id': '1oEepCQNgwsTFqyKzxD3ckw3zAYWLFoSL', 'name': 'autocam_s2'}
{'id': '13nAYMK4YVQe5DIZD6QHWcPQkEUeaYKT51CJTXcKeTaY', 'name': 'ISPRS Summer School 2024 - Program of Activities'}
{'id': '1QiufI0oUDlSVZrVPDwiixPH0Sve2gp8bEI0qbuNUKDo', 'name': 'backup AUTOCAM RRL NOTES'}
{'id': '1-hpNXQKX0aqaXo2BsdXjiUwqJGqPUuAT', 'name': '061424_AutoCAM-PhilSA-Meeting-Request.pdf'}

File Downloaded
